# Week 14

Scikit-Learn and regression modeling

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/data_utils.py
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/io_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from data_utils import MinMaxScaler, StandardScaler
from data_utils import PolynomialFeatures
from data_utils import LinearRegression, RandomForestClassifier
from data_utils import KMeansClustering, GaussianClustering, SpectralClustering
from data_utils import regression_error
from io_utils import object_from_json_url

## Regression

Regression, or Regression Analysis, is a set of statistical processes for estimating the relationship between a dependent variable (sometimes called the 'outcome', 'response' or 'label') and one or more independent variables (called 'features', 'dimensions' or 'columns').

For example, let's say we have the following data about people's wages and years of experience:

<img src="./imgs/wages-exp.png" width="620px"/>

We could use regression to calculate how the values for wages are affected by years of experience in our dataset, and then create a function to more generally estimate the relation between wages and experience:

<img src="./imgs/wages-exp-fit.png" width="620px"/>

We could now estimate wages for values of years of experience that we didn't have measurements for.

This is an estimate, but the more points we use and the more features we have in our dataset the better the regression results will be.

### Setting up Regression

For a simple dataset we can perform regression by following these steps:

1. Load dataset
2. Encode label features as numbers
3. Normalize the data
4. Separate the outcome variable and the feature variables
5. Create a regression model
6. Run model on input data and measure error

## Diamond Prices

Let's use the dataset from last week to set up a diamond price estimator.

Steps 1 - 3 should look familiar:

In [ ]:
## 1. Load Dataset
DIAMONDS_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/diamonds.json"

# Read into DataFrame
diamonds_data = object_from_json_url(DIAMONDS_FILE)
diamonds_df = pd.DataFrame.from_records(diamonds_data)


## 2. Encode non-numeric values
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

diamond_encoder = OrdinalEncoder(categories=[cut_order, color_order, clarity_order])

ccc_vals = diamond_encoder.fit_transform(diamonds_df[["cut", "color", "clarity"]].values)
diamonds_df[["cut", "color", "clarity"]] = ccc_vals


## 3. Normalize
diamond_scaler = MinMaxScaler()
diamonds_scaled = diamond_scaler.fit_transform(diamonds_df)

### Chose Features

Now we separate the outcome variable values and the independent variables.

Let's start simple and use only one feature: carat.

In [ ]:
## 4. Separate the outcome variable and the independent variables
prices = diamonds_scaled["price"]
carats = diamonds_scaled[["carat"]]

# Plot the variables, just for checking
plt.scatter(carats, prices, marker='o', linestyle='', alpha=0.3)
plt.xlabel("carat")
plt.ylabel("price")
plt.show()

### Model

Setup and create regression model:

In [ ]:
## 5. Create a LinearRegression object
price_model = LinearRegression()

# Create a model that relates price of diamonds to their carat value
model = price_model.fit(carats, prices)

### Evaluate

Run the regression model, put the result back in a DataFrame and measure its error.

In [ ]:
## 6. Run the model on the training data
predicted_scaled = price_model.predict(carats)

# Un-normalize the data
predicted = diamond_scaler.inverse_transform(predicted_scaled)

# Measure error
regression_error(diamonds_df["price"], predicted["price"])

### Result

Hmmm.... what this means is that on average our model is wrong by $\$1388$ dollars.

We can plot our predictions with the original data:

In [ ]:
# Plot the original values
plt.scatter(carats, prices, marker='o', linestyle='', alpha=0.3)
plt.xlabel("carat")
plt.ylabel("price")

# Plot the predictions
plt.scatter(carats, predicted_scaled, color='r', marker='o', linestyle='', alpha=0.05)
plt.xlabel("carat")
plt.ylabel("price")
plt.show()

### Interpretation

We're only using one variable to model the price using linear regression, so, as the name suggests, the resulting model is a line:

$\displaystyle price = \beta \cdot carat$

(This should look familiar to a line equation from algebra: $y = m \cdot x + b$)

$\beta$ in this equation is a constant calculated by the model. The model uses all of the data point values about price and carat to calculate this ONE constant that defines our line.

For every value of $carat$ the model gives us a value for $price$.

### Using more features

Let's use a few more features to build our model.

This time our model equation will have multiple independent variables:

$\displaystyle price = \beta_0 \cdot carat + \beta_1 \cdot width + \beta_2 \cdot length$

The $\beta_i$ values in this equation are constants that the model calculates. There are $3$ now, so the model has more parameters to adjust in order to get a better fit.

In [ ]:
## 4. Separate the outcome variable and the independent variables
prices = diamonds_scaled["price"]
features = diamonds_scaled[["carat", "x", "y"]]

## 5. Create a LinearRegression object
price_model = LinearRegression()

# Create a model that relates price of diamonds to their carat value as well as width and length
price_model.fit(features, prices)

## 6. Run the model on the training data
predicted_scaled = price_model.predict(features)

# Un-normalize the data
predicted = diamond_scaler.inverse_transform(predicted_scaled)

# Measure error
regression_error(diamonds_df["price"], predicted["price"])

### Result

This is better. The error decreased.

We can plot our new model, but since we are limited to $3$ physical dimensions that we can comprehend, we can't plot price as a function of all $3$ of our features.

We'll just look at how price varies along with the length and width of the diamonds:

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

xs = diamonds_df[["x"]].values
ys = diamonds_df[["y"]].values

ps = diamonds_df[["price"]].values
pps = predicted[["price"]].values

ax.scatter(xs, ys, ps, marker='o', linestyle='', alpha=0.1)
ax.scatter(xs, ys, pps, color='r', marker='o', linestyle='', alpha=0.1)

ax.set_xlabel('width')
ax.set_ylabel('length')
ax.set_zlabel('price')

plt.show()

### Using ALL features

Let's use all $9$ features from the dataset to build our model.

The model equation is something like:

$\displaystyle price = \beta_0 \cdot x_0 + \beta_1 \cdot x_1 + ... + \beta_8 \cdot x_8$

Where the $x_i$ values are the values of our features (carat, width, etc) and the $\beta_i$ parameters are the constant values that the model will calculate.


In [ ]:
## 4. Separate the outcome variable and the independent variables
prices = diamonds_scaled["price"]

# All except price
features = diamonds_scaled.drop(columns=["price"])

## 5. Create a LinearRegression object
price_model = LinearRegression()

# Create a model that relates price of diamonds to many features
price_model.fit(features, prices)

## 6. Run the model on the training data
predicted_scaled = price_model.predict(features)

# Un-normalize the data
predicted = diamond_scaler.inverse_transform(predicted_scaled)

# Measure error
regression_error(diamonds_df["price"], predicted["price"])

### Result

The error is getting better.

Let's sort and plot all of the prices from the original dataset and the reconstructed prices from our model:

In [ ]:
prices_original = diamonds_df["price"]
prices_predicted = predicted["price"]

# Plot the original and predicted prices
plt.plot(sorted(prices_original), marker='o', linestyle='', alpha=0.3)
plt.plot(sorted(prices_predicted), color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("price")
plt.show()

### Interpretation

The model doesn't look too bad.

It seems to not be performing very well for diamonds on the extreme ends of price: the too cheap and too expensive ones.

And since even a small percentage of error for an expensive diamond contributes to a large error in dollars, this is probably where a lot of the error is coming from.

### Even MORE Features !

One trick to improve our model is to create some extra features from the current ones.

For example, in addition to considering carat and width of each diamond separately, we can create a feature that is a combination of these two values.

Considering just those $2$ features, instead of having an equation that is like this:

$\displaystyle price = \beta_0 \cdot carat + \beta_1 \cdot width$

We can try to model an equation that has quadratic terms, like this:

$\displaystyle price = \beta_0 \cdot carat + \beta_1 \cdot width + \beta_2 \cdot carat^2 + \beta_3 \cdot width^2 + \beta_4 \cdot carat \cdot width$

Or even cubic terms:

$\displaystyle price = \beta_0 \cdot carat + \beta_1 \cdot width + \beta_2 \cdot carat^2 + \beta_3 \cdot width^2 + \beta_4 \cdot carat \cdot width + \beta_5 \cdot carat^3 + \beta_6 \cdot width^3$

This allows our model to figure out more complex relationships between the features and consider non-linear relationships between features and price (maybe price goes up proportional to the square of the width of the diamond).

SciKit-Learn has an object called [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) that helps us do exactly this. We just have to instantiate it and use it to create some extra features for us.

In [ ]:
## 4. Separate the outcome variable and the independent variables
prices = diamonds_scaled["price"]
features = diamonds_scaled.drop(columns=["price"])

## 4B. Create extra features
poly = PolynomialFeatures(degree=3, include_bias=False)
features_poly = poly.fit_transform(features)

## 5. Create a LinearRegression object
price_model = LinearRegression()

# Create a model that relates price of diamonds to many features
result = price_model.fit(features_poly, prices)

## 6. Run the model on the training data
predicted_scaled = price_model.predict(features_poly)

# Un-normalize the data
predicted = diamond_scaler.inverse_transform(predicted_scaled)

# Measure error
regression_error(diamonds_df["price"], predicted["price"])

### Result

This is significantly better than our original model.

Let's sort and plot the resulting prices:

In [ ]:
# Plot sorted prices
prices_original = sorted(diamonds_df["price"])
prices_predicted = sorted(predicted["price"])

# Plot the original and predicted prices
plt.plot(prices_original, marker='o', linestyle='', alpha=0.3)
plt.plot(prices_predicted, color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("price")
plt.show()

### More plots

And since we can't see in 4D or 5D yet, let's just plot price as a function of a few individual features.

In [ ]:
# Plot price vs carat, x, y and z
for feat in ["carat", "x", "y", "z"]:
  x = diamonds_df[feat]
  prices_original = diamonds_df["price"]
  prices_predicted = predicted["price"]

  # Plot the original and predicted prices
  plt.plot(x, prices_original, marker='o', linestyle='', alpha=0.3)
  plt.plot(x, prices_predicted, color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
  plt.xlabel(feat)
  plt.ylabel("price")
  plt.show()

# 🎉🍾

These look ok. We have a nice model for diamond prices.

## More Regression!

Let's repeat the previous exercise using a different dataset.

This one is for wine quality.

In [ ]:
## 1. Load Dataset
WINE_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/wines.json"

# Read into DataFrame
wines_data = object_from_json_url(WINE_FILE)
wines_df = pd.DataFrame.from_records(wines_data)

# Look at features: values, types, names, etc
wines_df.head()

In [ ]:
## 3. Normalize
wine_scaler = MinMaxScaler()
wines_scaled = wine_scaler.fit_transform(wines_df)

# Since this is a new dataset, let's just peek at its covariance matrix
wines_scaled.cov()["quality"].sort_values()

### Plot

Looks like alcohol, acidity, density and chlorides are the $4$ features that mostly contribute to the quality.

Let's just take a look at graphs of quality as a function of these $4$ features:

In [ ]:
# Plot quality vs alcohol and volatile acidity
for feat in ["alcohol", "acidity", "density", "chlorides"]:
  x = wines_df[feat]
  quality_original = wines_df["quality"]

  # Plot the original quality
  plt.plot(x, quality_original, marker='o', linestyle='', alpha=0.3)
  plt.xlabel(feat)
  plt.ylabel("quality")
  plt.show()

### Regression

Let's just use all of our features to run regression:

In [ ]:
## 4. Separate the outcome variable and the independent variables
quality = wines_scaled["quality"]
features = wines_scaled.drop(columns=["quality"])

## 5. Create a LinearRegression object
quality_model = LinearRegression()

# Create a model that relates quality of wines to many features
result = quality_model.fit(features, quality)

## 6. Run the model on the training data
predicted_scaled = quality_model.predict(features)

# Un-normalize the data
predicted = wine_scaler.inverse_transform(predicted_scaled)

# Measure error
regression_error(wines_df["quality"], predicted["quality"])

### Plot Results

On average our predictions are within $0.77$ points of the real quality values.

Let's take a look at some plots:

In [ ]:
# Plot quality vs alcohol and volatile acidity
for feat in ["alcohol", "acidity", "density", "chlorides"]:
  x = wines_df[feat]
  quality_original = wines_df["quality"]
  quality_predicted = predicted["quality"]

  # Plot the original quality
  plt.plot(x, quality_original, marker='o', linestyle='', alpha=0.3)
  plt.plot(x, quality_predicted, color='r', marker='o', linestyle='', alpha=0.3)
  plt.xlabel(feat)
  plt.ylabel("quality")
  plt.show()

### Interpretation

# 🤔

Hmm.... these could be better.

Our model wasn't able to capture the fact that the resulting quality should be a discrete value and not a number with decimals.

This is because our quality category is not continuous, and instead can only have particular discrete values.

Instead of trying to calculate continuous values for quality, our model should really be trying to put the wines in the right quality category.

Let's use a different type of model for this task.

Instead of learning how to predict a continuous value from the independent variables, like this:

<img src="./imgs/wages-exp-fit.png" width="620px"/>

Our model should learn how to place data points into discrete groups, like this:

<img src="./imgs/wages-exp-classes.png" width="620px"/>

## Classification

This is what's called a *classification* problem, or task.

Instead of trying to model the behavior of a continuous value, like price or temperature, a classification model tries to predict the correct *label* for given input data.

The steps for training a classification model are the same:

1. Load dataset
2. Encode label features as numbers
3. Normalize the data
4. Separate the outcome variable and the feature variables
5. Create a model
6. Run model on input data and test data, and measure error

Even though we are trying to predict labels for our data, we still have to encode all label/categorical features, whether they are input or output variables.

### Random ? Forest ?

The particular classification model we will use is called a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

The model gets its name from another type of model called a [Decision Tree](https://scikit-learn.org/stable/modules/tree.html). During training, Decision Trees learn to model our data using simple decision rules, in a process that is conceptually similar to the game [Twenty Questions](https://en.wikipedia.org/wiki/Twenty_questions).

It learns to bin our data by *asking* a series of yes/no, if/else, questions using our features:

<img src="./imgs/decision-trees.jpg" width="720px"/>

A Random Forest Classifier is a model that combines a bunch of Tree models that were trained with randomly selected subsets of our features.

In [ ]:
## 4. Separate the outcome variable and the independent variables

# We're still using scaled feature variables
features = wines_scaled.drop(columns=["quality"])

# But, now our quality variable will be unscaled,
# so it keeps its original values as labels 0 - 9
quality = wines_df["quality"]

## 5. Create a Classifier object
quality_model = RandomForestClassifier()

# Create a model that classifies quality of wines based on many features
result = quality_model.fit(features, quality)

## 6. Run the model on the training data
predicted = quality_model.predict(features)

# Measure error
regression_error(wines_df["quality"], predicted["quality"])

### Plot Results

Ohh, that's better. On average, the model misses the quality value by about $0.1$ points.

Let's take a look at some plots to confirm that the model captured the discrete-ness of our label:

In [ ]:
# Plot quality vs alcohol and volatile acidity
for feat in ["alcohol", "acidity", "density", "chlorides"]:
  x = wines_df[feat]
  quality_original = wines_df["quality"]
  quality_predicted = predicted["quality"]

  # Plot the original quality
  plt.plot(x, quality_original, marker='o', linestyle='', alpha=0.3)
  plt.plot(x, quality_predicted, color='r', marker='o', linestyle='', alpha=0.3)
  plt.xlabel(feat)
  plt.ylabel("quality")
  plt.show()

### Test Dataset

One thing that we didn't do before, but is an important step when training any kind of ML model, is to actually test our model on data that wasn't used in the training. This will tell us whether our model has actually learned patterns from the data or just memorized the data.

In [ ]:
## 1. Load Dataset
WINE_TEST_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/wines-test.json"

# Read into DataFrame
wines_test_data = object_from_json_url(WINE_TEST_FILE)
wines_test_df = pd.DataFrame.from_records(wines_test_data)

## 3. Normalize
wines_test_scaled = wine_scaler.transform(wines_test_df)

## 4. Separate the independent variables
features_test = wines_test_scaled.drop(columns=["quality"])

## 6. Run the model on the test data
predicted_test = quality_model.predict(features_test)

# Measure error
regression_error(wines_test_df["quality"], predicted_test["quality"])

### Interpretation

This is not great, but it's not bad. On average our model is within $0.55$ quality points of the real values.

It's better than the result we originally got from linear regression on the test data ($0.76$).

Let's take a look at some plots:

In [ ]:
# Plot quality vs alcohol and volatile acidity
for feat in ["alcohol", "acidity", "density", "chlorides"]:
  x = wines_test_df[feat]
  quality_original = wines_test_df["quality"]
  quality_predicted = predicted_test["quality"]

  # Plot the original quality
  plt.plot(x, quality_original, marker='o', linestyle='', alpha=0.3)
  plt.plot(x, quality_predicted, color='r', marker='o', linestyle='', alpha=0.3)
  plt.xlabel(feat)
  plt.ylabel("quality")
  plt.show()

### Conclusion

We have an *ok* model for predicting wine quality based on a few parameters.

## More Wine ! 🍷🍷🍷

Let's pretend we own an online wine store.

We already created a model that predicts quality based on a bunch of other properties of wines. We could use this model to figure out how much to pay suppliers for the wine, and how much to charge costumers.

But, maybe this "quality" feature might not be something we want to share with our costumers. It seems abstract and subjective and would require explanations about our data and our process, which could create confusion.

Using the six other features might also not be very useful for costumers who want to buy new wines that are similar to ones that they have previously liked.

What we can do is classify the wines into groups that take into account all of the features of the dataset, but present costumers with a more manageable amount of information.

This is called [clustering](https://en.wikipedia.org/wiki/Cluster_analysis), or cluster analysis. We'll use it to divide our wines in such a way that wines in the same group, or *cluster*, are more similar to each other than to wines in other clusters.

The clusters won't necessarily correlate to the features in our dataset, but will be computed using a combination of the features.

Clustering is an example of an *unsupervised* learning method.

### Supervised Learning

The models that we've trained so far for doing regression and classification are considered *supervised* models. During training we give the model our input features, but also provide it with the *correct* values for the output features. These output features tend to be human-labeled values, and are sometimes called the *supervisory signals*.

When fully-labeled training data is processed during training, we are hoping that the model learns to extrapolate what it *sees* in the labeled data to new, unseen, unlabeled instances of data with the same input features, but unknown output values.

#### Supervised Classification:
<img src="./imgs/classification-00.jpg" width="620px"/>

### Unsupervised Learning

Unlike supervised learning, unsupervised models learn patterns from unlabeled data. This means all of the features are considered input features, and there are no separate output features or signals. The idea is that by analyzing and processing data in specific ways, the model is able to build a concise representation of its features and create new ways of interpreting, visualizing or generating similar data.

We can use unsupervised learning models to explore new datasets and try to simplify our data before we do any kind of supervised learning.

The steps for training an unsupervised model should seem familiar:

1. Load dataset
2. Encode label features as numbers
3. Normalize the data
4. Select variables and features to be considered
5. Create a model
6. Run model on input data and test data, and measure error

Even though it looks familiar, the last step isn't very obvious. How do we measure error on a model that doesn't have a set of correct answers?

Maybe *error* is not the right term, but we'll see how to define *metrics* to score and measure our unsupervised models.

#### Unsupervised Clusterings:
Since there are no correct labels, both of the following clusterings are valid!

<img src="./imgs/clustering-00.jpg" width="620px"/>

<img src="./imgs/clustering-01.jpg" width="620px"/>

### Preparing Data

In [ ]:
## 1. Load Dataset
WINE_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/wines.json"

# Read into DataFrame
wines_data = object_from_json_url(WINE_FILE)
wines_df = pd.DataFrame.from_records(wines_data)

## 3. Normalize
wine_scaler = StandardScaler()
wines_scaled = wine_scaler.fit_transform(wines_df)

## 4. Select variables to be considered
##    We're gonna drop the quality features to avoid re-clustering by quality
features = wines_scaled.drop(columns=["quality"])

### Clusterings

We are going to look at three different methods for clustering our data.

#### [K-means Clustering](https://scikit-learn.org/stable/modules/clustering.html#k-means):
Tries to separate the data into $k$ groups with similar statistical properties. Requires the number of clusters to be determined beforehand, and the algorithm tries to minimize the difference between objects in a cluster.

#### [Gaussian Clustering](https://scikit-learn.org/stable/modules/mixture.html#mixture):
This is similar to K-means, but this model assumes that all features of our data can be modeled as Gaussian distributions.

#### [Spectral Clustering](https://scikit-learn.org/stable/modules/clustering.html#spectral-clustering):
When appropriate, this method automatically combines and removes a few of our features, before doing K-means clustering. This should always be as good as, or better than, regular K-means clustering.

In [ ]:
predicted = {}

## 5. Create Clustering objects
n_clusters = 4
km_model = KMeansClustering(n_clusters=n_clusters)
gm_model = GaussianClustering(n_clusters=n_clusters)
sc_model = SpectralClustering(n_clusters=n_clusters)

## 6. Run the model on the training data
predicted['kmeans'] = km_model.fit_predict(features)
predicted['gaussian'] = gm_model.fit_predict(features)
predicted['spectral'] = sc_model.fit_predict(features)

### Plots

Since we can't see in $4D$ or $5D$ yet, let's pick $2$ or $3$ variables to visualize our clusters against.

We can look at *covariances* again and pick the top $2$ or $3$ variables related to `quality`.

In [ ]:
## Look at covariances again
wines_scaled.cov()["quality"].sort_values()

In [ ]:
# For plotting
xl, yl, zl = "alcohol", "chlorides", "density"
x = wines_scaled[xl]
y = wines_scaled[yl]
z = wines_scaled[zl]

for k,v in predicted.items():
  plt.scatter(x, z, c=v["clusters"], marker='o', linestyle='', alpha=0.5)
  plt.title(k)
  plt.xlabel(xl)
  plt.ylabel(zl)
  plt.xlim(-2.2, 3.2)
  plt.ylim(-2.5, 3.5)
  plt.show()

for k,v in predicted.items():
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(projection='3d')

  ax.scatter(x, y, z, c=v, marker='o', linestyle='', alpha=0.5)

  ax.set_title(k)
  ax.set_xlabel(xl)
  ax.set_ylabel(yl)
  ax.set_zlabel(zl)

  ax.set_ylim(-2.5, 8)
  ax.set_zlim(-2.5, 2.5)

  plt.show()

### Scoring

Would be nice to have a way to measure how good the clusters are.

It would help determine if we need more clusters, or if one method is actually better than the others.

There are a couple of ways to do this. We'll look at two of them.

### Distance

The first kind of scoring uses the distances between each point and its cluster's center as a metric.

This is sometimes called the L2-distance, and it's just like the more familiar [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) from geometry, but extended to measure more than just $2$ or $3$ dimensions/parameters.

Smaller values mean that the cluster center is a good representation of its members.

In [ ]:
km_model.distance_error(), gm_model.distance_error(), sc_model.distance_error()

### Likelihood

The second kind of scoring treats each cluster as a potential normal distribution and then calculates the likelihood that each point came from its cluster distribution.

Values closer to zero mean that the clusters' statistical properties (mean, variation) are good estimators for the data.

In [ ]:
km_model.likelihood_error(), gm_model.likelihood_error(), sc_model.likelihood_error()

### Balance

A final metric we can consider when analyzing different clustering algorithms and strategies is to see how balanced the resulting clusters are.

This might not be very important in some cases, but in other cases where we might be trying to distribute something equally among a population, this is a good metric to look at.

We can compute it by seeing how far our cluster counts are from a balanced clustering with equal size clusters.

In [ ]:
# If we have a list of clusters, like this:
print(predicted['kmeans'][:10])

# This gives us the counts for each label:
label_counts = predicted['kmeans']['clusters'].value_counts()
print(label_counts)

# And this gives an idea of how far they are from a fully-balanced clustering
(label_counts / len(predicted['kmeans']['clusters']) - (1 / n_clusters)).abs().max()

### Balance Error

Luckily this has been implemented for us and we can get our clustering model balance error like this:

In [ ]:
km_model.balance_error(), gm_model.balance_error(), sc_model.balance_error()

### Number of clusters

By these metrics, it seems like the Spectral Clustering algorithm performs a bit better, even though it doesn't produce the most balanced clusters.

Let's try different cluster numbers and see if there's a *better* number of clusters we can use:

In [ ]:
num_clusters = list(range(1,8))

dist_err = []
like_err = []
bala_err = []

for n in num_clusters:
  mm = SpectralClustering(n_clusters=n)
  mm.fit_predict(features)
  dist_err.append(mm.distance_error())
  like_err.append(mm.likelihood_error())
  bala_err.append(mm.balance_error())


plt.plot(num_clusters, dist_err, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Distance Error")
plt.show()

plt.plot(num_clusters, like_err, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Likelihood Error")
plt.show()

plt.plot(num_clusters, bala_err, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Balance Error")
plt.show()

### Interpretation

Looks like $4$ could be good number of clusters for this model.

Let's look at some graphs:

In [ ]:
predicted = {}

for n in [3,4,5]:
  m_model = SpectralClustering(n_clusters=n)
  predicted["Spectral %s" % n] = m_model.fit_predict(features)

# For plotting
xl, yl, zl = "alcohol", "chlorides", "density"
x = wines_scaled[xl]
y = wines_scaled[yl]
z = wines_scaled[zl]

for k,v in predicted.items():
  plt.scatter(x, z, c=v["clusters"], marker='o', linestyle='', alpha=0.5)
  plt.title(k)
  plt.xlabel(xl)
  plt.ylabel(zl)
  plt.xlim(-2.2, 3.2)
  plt.ylim(-2.5, 3.5)
  plt.show()

for k,v in predicted.items():
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(projection='3d')

  ax.scatter(x, y, z, c=v, marker='o', linestyle='', alpha=0.5)

  ax.set_title(k)
  ax.set_xlabel(xl)
  ax.set_ylabel(yl)
  ax.set_zlabel(zl)

  ax.set_ylim(-2.5, 8)
  ax.set_zlim(-2.5, 2.5)

  plt.show()